In [1]:
#formato binario
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
def obtener_datos():
    # use creds to create a client to interact with the Google Drive API
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(creds)
    # Find a workbook by name and open the first sheet
    # Make sure you use the right name here.
    sheet = client.open("DatosTFG_SistemasRecomendacion").sheet1

    # Extract and print all of the values
    list_of_hashes = sheet.get_all_records()
    #print(list_of_hashes)
    tabla = pd.DataFrame(data=sheet.get_all_records())
    return tabla

In [3]:
def transfor_datos(tabla):
#almacenar en un diccionario de diccionarios cuya clave sea el usuario, y como valor un conjunto de  diccionarios
#(clave=nombre asignatura, valor=ponderacion). No introducimos el Token ni la fecha en la que se realizó el cuestionario. 
    asignaturas= list(tabla.columns)
    diccAsi={}
    diccAsi.setdefault(1,{})
    for k in asignaturas:
        if k != 'Token'and k!='Submitted At':
            for i, j in zip(tabla[k], range(len(tabla['Algoritmia'])) ):
                diccAsi.setdefault(j,{})
                diccAsi[j][k]=i
    tabla1 = pd.DataFrame.from_dict(diccAsi)
    tabla1= tabla1.T
    #transformamos los vacíos en NaN para evitar el error de calcular el coeficiente de correlación de pearson entre un int y un string ('' equivale a un str)
    tabla1= tabla1.replace('', np.nan, regex=True)
    return tabla1

In [4]:
#formato binario
def recuperarDatos(nombreArchivo):
    archivo = open(nombreArchivo, "rb")
    tablaDatos = pickle.load(archivo)
    archivo.close()
    return tablaDatos

In [5]:
tabla1= recuperarDatos('archivoDatos.bin')
tabla1

,Algoritmia,Análisis y Diseño de Sistemas,Aplicaciones de Bases de Datos,Arquitectura de Computadores,Arquitecturas Paralelas,Bases de Datos,Computación Neuronal y Evolutiva,Control por Computador,Cálculo,Desarrollo Avanzado de Sistemas Software,...,Programación Concurrente y de Tiempo Real,Programación de Sistemas Operativos,Redes,Seguridad Informática,Sistemas Distribuidos,Sistemas Empotrados y de Tiempo Real,Sistemas Inteligentes,Sistemas Operativos,Validación y Pruebas,Álgebra Lineal
0,5,4,5,5,5,5,5.0,NaN,5,NaN,...,5,NaN,4,3,4.0,NaN,5,2,NaN,3
1,4,1,5,4,1,5,3.0,NaN,1,NaN,...,2,5.0,5,1,5.0,NaN,4,1,5.0,3
2,1,1,5,1,1,4,NaN,5.0,5,NaN,...,1,NaN,1,5,NaN,NaN,4,5,5.0,1
3,5,2,5,4,4,5,4.0,NaN,4,NaN,...,4,4.0,4,3,NaN,NaN,4,3,NaN,4
4,5,1,5,5,5,5,NaN,NaN,4,5.0,...,5,NaN,4,1,5.0,NaN,5,1,NaN,2
5,5,3,5,4,4,4,NaN,NaN,1,NaN,...,4,NaN,2,4,NaN,NaN,5,3,NaN,2
6,4,1,4,5,4,5,3.0,NaN,3,NaN,...,3,NaN,1,2,NaN,NaN,5,3,4.0,1
7,4,1,1,5,3,4,4.0,NaN,2,5.0,...,3,NaN,2,3,3.0,NaN,5,4,5.0,3
8,4,3,4,5,4,5,NaN,NaN,4,NaN,...,4,NaN,3,4,NaN,NaN,4,4,NaN,2
9,3,2,3,5,1,3,NaN,NaN,3,NaN,...,3,NaN,1,3,NaN,NaN,3,1,NaN,1


In [6]:
#tabla1= obtener_datos()

In [7]:
tabla1= tabla1.replace('', np.nan, regex=True)
tabla1=transfor_datos(tabla1)
tabla1

,Algoritmia,Análisis y Diseño de Sistemas,Aplicaciones de Bases de Datos,Arquitectura de Computadores,Arquitecturas Paralelas,Bases de Datos,Computación Neuronal y Evolutiva,Control por Computador,Cálculo,Desarrollo Avanzado de Sistemas Software,...,Programación Concurrente y de Tiempo Real,Programación de Sistemas Operativos,Redes,Seguridad Informática,Sistemas Distribuidos,Sistemas Empotrados y de Tiempo Real,Sistemas Inteligentes,Sistemas Operativos,Validación y Pruebas,Álgebra Lineal
0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0,NaN,...,5.0,NaN,4.0,3.0,4.0,NaN,5.0,2.0,NaN,3.0
1,4.0,1.0,5.0,4.0,1.0,5.0,3.0,NaN,1.0,NaN,...,2.0,5.0,5.0,1.0,5.0,NaN,4.0,1.0,5.0,3.0
2,1.0,1.0,5.0,1.0,1.0,4.0,NaN,5.0,5.0,NaN,...,1.0,NaN,1.0,5.0,NaN,NaN,4.0,5.0,5.0,1.0
3,5.0,2.0,5.0,4.0,4.0,5.0,4.0,NaN,4.0,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,4.0,3.0,NaN,4.0
4,5.0,1.0,5.0,5.0,5.0,5.0,NaN,NaN,4.0,5.0,...,5.0,NaN,4.0,1.0,5.0,NaN,5.0,1.0,NaN,2.0
5,5.0,3.0,5.0,4.0,4.0,4.0,NaN,NaN,1.0,NaN,...,4.0,NaN,2.0,4.0,NaN,NaN,5.0,3.0,NaN,2.0
6,4.0,1.0,4.0,5.0,4.0,5.0,3.0,NaN,3.0,NaN,...,3.0,NaN,1.0,2.0,NaN,NaN,5.0,3.0,4.0,1.0
7,4.0,1.0,1.0,5.0,3.0,4.0,4.0,NaN,2.0,5.0,...,3.0,NaN,2.0,3.0,3.0,NaN,5.0,4.0,5.0,3.0
8,4.0,3.0,4.0,5.0,4.0,5.0,NaN,NaN,4.0,NaN,...,4.0,NaN,3.0,4.0,NaN,NaN,4.0,4.0,NaN,2.0
9,3.0,2.0,3.0,5.0,1.0,3.0,NaN,NaN,3.0,NaN,...,3.0,NaN,1.0,3.0,NaN,NaN,3.0,1.0,NaN,1.0


In [15]:
def coef_corr_pearson(dataframe,a,b,rows=True):
    dataframe
    if rows:
        common_ratings= dataframe.ix[[a,b]].dropna(axis=1).as_matrix()
        a_average = dataframe.ix[a].mean()
        b_average = dataframe.ix[b].mean()       
    else:
        common_ratings= dataframe[[a,b]].dropna(axis=0).as_matrix().T
        a_average=dataframe[a].mean()
        b_average=dataframe[b].mean()              
    if common_ratings.any(): 
        a_common_ratings=common_ratings[0,:]
        b_common_ratings=common_ratings[1,:]
        pcorr = np.dot(a_common_ratings-a_average,b_common_ratings-b_average) / ( 
                 np.sqrt(np.dot(a_common_ratings-a_average,a_common_ratings-a_average))*
                 np.sqrt(np.dot(b_common_ratings-b_average,b_common_ratings-b_average)))
        if np.isnan(pcorr):
            return 0
        else:
            return pcorr
    else:
        return 0

In [16]:
def similarity_matrix_for_items(data,similarity=coef_corr_pearson):
    # Columns: items
    # Rows: users
    matrix = {}
    for item1 in data.columns.tolist():
        other_items = data.columns.tolist()[data.columns.tolist().index(item1):len(data.columns.tolist())]
        other_items
        #similarity_matrix_for_items
        matrix[item1]=dict([(item2,similarity(data,item1,item2,rows=False)) for item2 in other_items])
    for item1 in data.columns.tolist():
        other_items = data.columns.tolist()[data.columns.tolist().index(item1)+1:len(data.columns.tolist())]
        for item2 in other_items:
            matrix[item2][item1] = matrix[item1][item2]
    return matrix


In [17]:
def calculaPrediccion(dataframe,matrizSim,asign,usuario):
    # Encontrar los k usuarios que evaluaron 'asign'
    kUsuarios=np.where(dataframe[asign] > 0)[0]
    # Calcular la similitud de esos k usuarios a 'usuario'
    k_Similitu=matrizSim[usuario][kUsuarios]

    # Unir índices de usuarios y similitudes
    k_zip=list(zip(k_Similitu,kUsuarios))

    # Media valoraciones de usuario (sobre asignaturas cursadas)
    user_u_average=dataframe.T[usuario][list(np.where(dataframe.T[usuario]>0)[0])].mean()

    # Media de los valores absolutos de similitud de vecions próximos a usuario
    k_sim=[k_zip[i][0] for i in range(len(k_zip))]
    k_sim_average=np.abs(np.array(k_sim)).sum()
    
    # Estimación del rating a la asignatura asign
    rating_user_u_item_i=0
    for user_v in [k_zip[i][1] for i in range(len(k_zip))]:
        rating_user_u_item_i+= (dataframe.T[user_v][asign] - dataframe.T[user_v][list(np.where(dataframe.T[user_v]>0)[0])].mean())*matrizSim[usuario][user_v]
    rating_user_u_item_i=user_u_average + rating_user_u_item_i/k_sim_average
    #Devolvemos el rating estimado
    return rating_user_u_item_i

In [18]:
dataframe=tabla1
matrizSim=pd.DataFrame(similarity_matrix_for_items(tabla1.T))


In [19]:
asign="Computación Neuronal y Evolutiva"
usuario=2
calculaPrediccion(dataframe,matrizSim,asign,usuario)

3.6086914992410231

In [20]:
type(tabla1["Aplicaciones de Bases de Datos"][5])

numpy.float64

In [21]:
tabla1

,Algoritmia,Análisis y Diseño de Sistemas,Aplicaciones de Bases de Datos,Arquitectura de Computadores,Arquitecturas Paralelas,Bases de Datos,Computación Neuronal y Evolutiva,Control por Computador,Cálculo,Desarrollo Avanzado de Sistemas Software,...,Programación Concurrente y de Tiempo Real,Programación de Sistemas Operativos,Redes,Seguridad Informática,Sistemas Distribuidos,Sistemas Empotrados y de Tiempo Real,Sistemas Inteligentes,Sistemas Operativos,Validación y Pruebas,Álgebra Lineal
0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0,NaN,...,5.0,NaN,4.0,3.0,4.0,NaN,5.0,2.0,NaN,3.0
1,4.0,1.0,5.0,4.0,1.0,5.0,3.0,NaN,1.0,NaN,...,2.0,5.0,5.0,1.0,5.0,NaN,4.0,1.0,5.0,3.0
2,1.0,1.0,5.0,1.0,1.0,4.0,NaN,5.0,5.0,NaN,...,1.0,NaN,1.0,5.0,NaN,NaN,4.0,5.0,5.0,1.0
3,5.0,2.0,5.0,4.0,4.0,5.0,4.0,NaN,4.0,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,4.0,3.0,NaN,4.0
4,5.0,1.0,5.0,5.0,5.0,5.0,NaN,NaN,4.0,5.0,...,5.0,NaN,4.0,1.0,5.0,NaN,5.0,1.0,NaN,2.0
5,5.0,3.0,5.0,4.0,4.0,4.0,NaN,NaN,1.0,NaN,...,4.0,NaN,2.0,4.0,NaN,NaN,5.0,3.0,NaN,2.0
6,4.0,1.0,4.0,5.0,4.0,5.0,3.0,NaN,3.0,NaN,...,3.0,NaN,1.0,2.0,NaN,NaN,5.0,3.0,4.0,1.0
7,4.0,1.0,1.0,5.0,3.0,4.0,4.0,NaN,2.0,5.0,...,3.0,NaN,2.0,3.0,3.0,NaN,5.0,4.0,5.0,3.0
8,4.0,3.0,4.0,5.0,4.0,5.0,NaN,NaN,4.0,NaN,...,4.0,NaN,3.0,4.0,NaN,NaN,4.0,4.0,NaN,2.0
9,3.0,2.0,3.0,5.0,1.0,3.0,NaN,NaN,3.0,NaN,...,3.0,NaN,1.0,3.0,NaN,NaN,3.0,1.0,NaN,1.0
